In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
import math
import importlib
import LSTM_model
importlib.reload(LSTM_model)
from LSTM_model import ShallowLSTMNet
from torchinfo import summary

In [2]:
in_chans = 22
n_classes = 4
input_window_samples = 1125
model = ShallowLSTMNet(in_chans,n_classes,input_window_samples)

In [3]:
print(model)

ShallowLSTMNet(
  (lstm): LSTM(22, 32, batch_first=True, bidirectional=True)
  (spatial): Conv1d(64, 40, kernel_size=(25,), stride=(1,), padding=(12,))
  (pool): AvgPool1d(kernel_size=(50,), stride=(50,), padding=(0,))
  (batch_norm): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=880, out_features=4, bias=True)
)


In [4]:
summary(model,input_size=(2, 22, 1125),col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowLSTMNet                           [2, 22, 1125]             [2, 4]                    --                        --
├─LSTM: 1-1                              [2, 1125, 22]             [2, 1125, 64]             14,336                    --
├─Conv1d: 1-2                            [2, 64, 1125]             [2, 40, 1125]             64,040                    [25]
├─BatchNorm1d: 1-3                       [2, 40, 1125]             [2, 40, 1125]             80                        --
├─AvgPool1d: 1-4                         [2, 40, 1125]             [2, 40, 22]               --                        [50]
├─Dropout: 1-5                           [2, 880]                  [2, 880]                  --                        --
├─Linear: 1-6                            [2, 880]                  [2, 4]                    3,524                     --
Total para

In [9]:
model = torch.load("ShallowLSTM_52_3.pth",weights_only=False,map_location=torch.device('cpu'))
print("Dropout:", model.dropout.p)  # dropout rate
print("Input channels (n_chans):", model.lstm.input_size)

# LSTM parameters
print("LSTM hidden size:", model.lstm.hidden_size)
print("LSTM num layers:", model.lstm.num_layers)
print("LSTM bidirectional:", model.lstm.bidirectional)

# Conv1d (spatial) kernel info
print("Conv1d kernel size:", model.spatial.kernel_size)
print("Conv1d out_channels (num_kernels):", model.spatial.out_channels)

# Pooling
print("Pooling kernel size (pool_size):", model.pool.kernel_size)

# Fully connected
print("Final Linear input size:", model.fc.in_features)
print("Final Linear output size (n_outputs):", model.fc.out_features)

Dropout: 0.5
Input channels (n_chans): 32
LSTM hidden size: 10
LSTM num layers: 1
LSTM bidirectional: True
Conv1d kernel size: (50,)
Conv1d out_channels (num_kernels): 40
Pooling kernel size (pool_size): (10,)
Final Linear input size: 1600
Final Linear output size (n_outputs): 3
